In [3]:
#pip install tmdbsimple
import tmdbsimple as tmdb
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import pickle

tmdb.API_KEY = 'ecb4864639691c0b1573d1cdaf39a336'
path_data = 'C:/root/Universita/project_dmdv/data/'

### Movies

In [32]:
#elenco id dell'estrazione giornaliera
movies_ids = list(pd.read_json(path_data + 'tmdb/daily_file/movie_ids_daily.json',
                               lines = True, encoding = 'UTF-8')['id'])
len(movies_ids)

481127

In [34]:
#inizializzazione liste di dataframe
movies = []
cast_movies = []
crew_movies = []
j = 1
for id_ in movies_ids:
    try:
        #creazione id
        my_id = 'm-' + str(j)
        j += 1
        #interrogazione dell'api
        query = tmdb.Movies(id_)
        current_row = pd.DataFrame([query.info()])
        current_row['movie_id'] = my_id
        #concateno alla lista di dataframe
        movies = movies + [current_row]
        
        #controllo se c'e la lista di persone del cast, se c'e concateno alla lista di dataframe
        credits = query.credits()
        cast = credits['cast']
        if len(cast) > 0:
            cast = pd.DataFrame(cast)
            cast['tmdb_movie_id'] = id_
            cast['movie_id'] = my_id
            cast_movies = cast_movies + [cast]
        
        #controllo se c'e la lista di persone della crew, se c'e concateno alla lista di dataframe
        crew = credits['crew']
        if len(crew) > 0:
            crew = pd.DataFrame(crew)
            crew['tmdb_movie_id'] = id_
            crew['movie_id'] = my_id
            crew_movies = crew_movies + [crew]
            
    except:
        #se va in errore stampo l'id del film
        print(id_)

#concateno i dataframe e scrivo il file json
pd.concat(movies).reset_index().to_json(path_data + 'tmdb/json/tmdb_movies.json')
pd.concat(cast_movies).reset_index().to_json(path_data + 'tmdb/json/tmdb_movies_cast.json')
pd.concat(crew_movies).reset_index().to_json(path_data + 'tmdb/json/tmdb_movies_crew.json')

### TV Series

In [4]:
len(tv_series_ids)

95877

In [3]:
tv_series_ids = list(pd.read_json(path_data + 'tmdb/daily_file/tv_series_ids_daily.json',
                                 lines = True, encoding = 'UTF-8')['id'])

In [31]:
#inizializzazione liste di dataframe
tvs = []
cast_tvs = []
crew_tvs = []
seasons_tvs = []
j = 1
for id_ in tv_series_ids:
    
    #creazione dell'id
    my_id = 'tv-' + str(j)
    j+=1
    #interrogazione api tmdb
    query = tmdb.TV(id_)
    try:
        current_row = pd.DataFrame([query.info()])
    except:
        #se va in errore stampo l'id e passo all'iterazione successiva
        print('Series: ' + str(id_))
        continue
    current_row['series_id'] = my_id
    #concateno alla lista di dataframe
    tvs = tvs + [current_row]
    
    credits = query.credits()
    #controllo se c'e la lista di persone del cast, se c'e concateno alla lista di dataframe
    cast = credits['cast']
    if len(cast) > 0:
        cast = pd.DataFrame(cast)
        cast['tmdb_series_id'] = id_
        cast['series_id'] = my_id
        cast_tvs = cast_tvs + [cast]

    #controllo se c'e la lista di persone della crew, se c'e concateno alla lista di dataframe
    crew = credits['crew']
    if len(crew) > 0:
        crew = pd.DataFrame(crew)
        crew['tmdb_series_id'] = id_
        crew['series_id'] = my_id
        crew_tvs = crew_tvs + [crew]
    
    #faccio passare tutte le stagioni presenti nel campo seasons
    seasons = current_row['seasons'].iloc[0]
    for s in seasons:
        try:
            #query tmdb per id serie e numero stagione
            current_df = pd.DataFrame([tmdb.TV_Seasons(id_, s['season_number']).info()])
            current_df['series_id'] = my_id
            current_df['tmdb_series_id'] = id_
            seasons_tvs = seasons_tvs + [current_df]
        except:
            #se va in errore stampo l'id e il numero stagione
            print('Series: ' + str(id_) + '    Season: ' + str(s))

#concateno i dataframe e scrivo i json
pd.concat(tvs).reset_index().to_json(path_data + 'tmdb/json/tmdb_tvs.json')
pd.concat(cast_tvs).reset_index().to_json(path_data + 'tmdb/json/tmdb_tvs_cast.json')
pd.concat(crew_tvs).reset_index().to_json(path_data + 'tmdb/json/tmdb_tvs_crew.json')
pd.concat(seasons_tvs).reset_index().to_json(path_data + 'tmdb/json/tmdb_tv_seasons.json')

### People

In [37]:
people_ids = list(pd.read_json(path_data + 'tmdb/daily_file/person_ids_daily.json',
                                 lines = True, encoding = 'UTF-8')['id'])

In [42]:
len(people_ids)

1502610

In [39]:
#inizializzazione liste di dataframe
people = []
failed = []
j = 1
for id_ in people_ids:
    try:
        #query tmdb
        current_row = pd.DataFrame([tmdb.People(id_).info()])
        #creazione id
        current_row['person_id'] = 'p-' + str(j)
        j += 1
        #concateno alla lista di dataframe
        people = people + [current_row]
    except:
        #se va in errore stampo l'id
        print(id_)
#concateno i dataframe e scrivo il file .json
pd.concat(people).reset_index().to_json(path_data + 'tmdb/json/tmdb_people.json')